In [2]:
import pandas as pd
import numpy as np
import tejapi
import matplotlib.pyplot as plt
import matplotlib.transforms as transforms
from matplotlib.font_manager import FontProperties
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS'] # 解決MAC電腦 plot中文問題
plt.rcParams['axes.unicode_minus'] = False
tejapi.ApiConfig.api_key ="GI3jyGfk4dTa3FcmcPi0nc9nBNlXZt"
tejapi.ApiConfig.api_base="http://10.10.10.66"
tejapi.ApiConfig.ignoretz = True

In [3]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 籌碼集中度

In [4]:
df = tejapi.get('TWN/AMTOP1',  #從TEJ api撈取所需要的資料
                chinese_column_name = True,
                paginate = True,
                coid='2330',
                mdate = {'gt':'2017-01-01'},
                opts={'columns':['coid','mdate', 'key3','buy', 'sell', 'total']})

In [5]:
df1 = tejapi.get('TWN/APRCD1',  #從TEJ api撈取所需要的資料
                chinese_column_name = True,
                paginate = True,
                coid='2330',
                mdate = {'gt':'2017-01-01'},
                opts={'columns':['coid','mdate', 'volume', 'close_adj']})

In [6]:
df.dropna(inplace=True)
df.sort_values(['日期','買進股數'], ascending=[1,0], inplace=True)#根據買進股數由大到小排序
df_buy = df.groupby(['日期']).head(15)#取出每個日期的前15筆

In [8]:
df_buy

,證券名稱,日期,證券碼,買進股數,賣出股數,成交金額
None,,,,,,
5,2330,2017-01-03,000144 美林證券,9869.0,1041.0,1992902
6,2330,2017-01-03,000147 台灣摩根士丹,2432.0,5189.0,1391968
23,2330,2017-01-03,000930 華南永昌證券,1266.0,95.0,248522
20,2330,2017-01-03,000888 國泰證券,895.0,294.0,217245
19,2330,2017-01-03,000884 玉山證券,808.0,86.0,163188
...,...,...,...,...,...,...
54218,2330,2022-11-22,000116 日盛證券,929.0,751.0,818717
54233,2330,2022-11-22,000844 摩根大通,897.0,1173.0,1001739
54243,2330,2022-11-22,0009A0 永豐金證券,888.0,1406.0,1112760


In [9]:
result = pd.DataFrame(index=df_buy['日期'].unique())#創一個新的表，日期當index
result['買進股數'] = df_buy.groupby(['日期'])['買進股數'].sum()# 把每個日期的前15筆買進股數加總

In [11]:
df.sort_values(['日期','賣出股數'], ascending=[1,0], inplace=True)#根據賣出股數由大到小排序
df_sell  = df.groupby(['日期']).head(15)#取出每個日期的前15筆
result['賣出股數'] = df_sell.groupby(['日期'])['賣出股數'].sum()

In [14]:
result_60 = result.rolling(60).sum() #根據n日做買進/賣出rolling 總和
result_60.rename(columns = {'買進股數':'買進股數_60', '賣出股數':"賣出股數_60"}, inplace=True)
result_120 = result.rolling(120).sum() #根據n日做買進/賣出rolling 總和
result_120.rename(columns = {'買進股數':'買進股數_120', '賣出股數':"賣出股數_120"}, inplace=True)

In [15]:
df1.set_index('年月日', inplace=True)

In [16]:
df1['成交量(千股)_60'] = df1['成交量(千股)'].rolling(60).sum()
df1['成交量(千股)_120'] = df1['成交量(千股)'].rolling(120).sum()
result2 = pd.merge(result_60, df1[['成交量(千股)_60', '成交量(千股)_120','收盤價(元)']], left_index=True, right_index=True) #把成交量跟買進/賣出合起來
result3 = pd.merge(result2, result_120, left_index=True, right_index=True) #把成交量跟買進/賣出合起來

In [19]:
result3['60日籌碼集中度'] = ((result3['買進股數_60'] - result3['賣出股數_60'])/result2['成交量(千股)_60'])*100 
result3['120日籌碼集中度'] = ((result3['買進股數_120'] - result3['賣出股數_120'])/result2['成交量(千股)_120'])*100 
# ((買方前15名近n日總買超量 - 賣方前15名近n日總賣超量) / 近n日總成交量)*100

In [67]:
result3

,買進股數_60,賣出股數_60,成交量(千股)_60,成交量(千股)_120,收盤價(元),買進股數_120,賣出股數_120,60日籌碼集中度,120日籌碼集中度
2017-01-03,NaN,NaN,NaN,NaN,153.2254,NaN,NaN,NaN,NaN
2017-01-04,NaN,NaN,NaN,NaN,153.2254,NaN,NaN,NaN,NaN
2017-01-05,NaN,NaN,NaN,NaN,153.6440,NaN,NaN,NaN,NaN
2017-01-06,NaN,NaN,NaN,NaN,154.0627,NaN,NaN,NaN,NaN
2017-01-09,NaN,NaN,NaN,NaN,154.0627,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2022-11-16,1926776.0,2047069.0,2509283.141,4481897.511,487.0000,3473275.0,3626523.0,-4.793919,-3.419266
2022-11-17,1948950.0,2064906.0,2535136.209,4492268.015,485.0000,3476553.0,3636937.0,-4.573955,-3.570223
2022-11-18,1966154.0,2077879.0,2557142.047,4447306.477,487.0000,3427260.0,3592232.0,-4.369135,-3.709481
2022-11-21,1982528.0,2095393.0,2577497.145,4444945.039,482.0000,3424072.0,3590536.0,-4.378860,-3.745018


In [175]:
# fig, ax1= plt.subplots(figsize =(20,16))
# plt.plot(result2.index , result2['收盤價(元)'],lw=1.5, label = '收盤價(元)')       
# plt.xlabel('日期',fontsize=15)
# plt.ylabel('收盤價', fontsize=15)
# plt.xticks(fontsize=15)
# plt.yticks(fontsize=15)
# plt.title('籌碼集中度',  fontsize=20)
# plt.legend(loc=1, fontsize=15) 

# ax2 = ax1.twinx() #跟第一張ax1的x軸一樣
# plt.plot(result2.index, result2['concentration'] , lw=1.5, color='r', label='籌碼集中度')
# plt.ylabel('籌碼集中度', fontsize=15)
# plt.yticks(fontsize=15)
# plt.legend(loc=2, fontsize=15)

# plt.gcf().autofmt_xdate() #讓x軸的時間軸比較寬鬆、漂亮
# plt.show()

In [21]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x = result3.index, y = result3['收盤價(元)'],  
               marker_color = 'rgb(220,187, 97)', name = '收盤價(元)', line = dict(width = 2)),
    secondary_y=True,) 

fig.add_trace(
    go.Scatter(x = result3.index, y = result3['60日籌碼集中度'],
                marker_color = 'rgb(219,113,153)', name = '60日籌碼集中度'), 
            secondary_y=False,)

fig.add_trace(
    go.Scatter(x = result3.index, y = result3['120日籌碼集中度'],
                marker_color = 'rgb(97, 137, 168)', name = '120日籌碼集中度'), 
            secondary_y=False,)

fig.update_yaxes(title_text= '籌碼集中度') 
fig.update_yaxes(title_text= '收盤價',secondary_y=True) 

fig.update_xaxes(
          showgrid=True,
          tickformat="%Y/%m/%d",
          title_text = '日期',
          titlefont = dict(size = 15))

fig.update_layout(
    title_text="籌碼集中度",
    width = 1200,
    height = 600,
    legend = dict(font = dict(size = 20)),
    legend_title = dict(font = dict(size = 20)),
    xaxis = dict(tickfont = dict(size=15)),
)

fig.show()

# 買進集中度/賣出集中度/差值

In [22]:
df2 = tejapi.get('TWN/ABSR', #從TEJ api撈取所需要的資料
                  chinese_column_name = True,
                  paginate = True,
                  mdate = {'gt':'2021-06-30', 'lt':'2022-11-22'},
                  coid="2330",
                  opts={'columns':['coid','mdate','key3','brk_nm','buy_s','sell_s']})

In [64]:
df2

,公司代碼,年月日,券商代碼,券商名稱,買進股數(股),賣出股數(股)
None,,,,,,
0,2330,2021-07-01,1020,合庫,3480,6570
1,2330,2021-07-01,1021,合庫台中,3755,3320
2,2330,2021-07-01,1022,合庫台南,2500,2404
3,2330,2021-07-01,1023,合庫高雄,1265,5200
4,2330,2021-07-01,1024,合庫嘉義,6144,4425
...,...,...,...,...,...,...
290760,2330,2022-11-21,9A9W,永豐市政,17327,12368
290761,2330,2022-11-21,9A9X,永豐竹科,8666,25337
290762,2330,2022-11-21,9A9x,永豐桃園,16044,31701


In [36]:
df2_buy = df2.sort_values(['年月日','買進股數(股)'], ascending=[1,0]).reset_index(drop=True)
df2_sell = df2.sort_values(['年月日','賣出股數(股)'], ascending=[1,0]).reset_index(drop=True)
df2_buy['排名'] = df2_buy.groupby('年月日')['買進股數(股)'].apply(lambda x: x.rank(method='dense' ,ascending=False))
df2_sell['排名'] = df2_sell.groupby('年月日')['賣出股數(股)'].apply(lambda x: x.rank(method='dense' ,ascending=False))

In [39]:
df2_buy['買進張數'] = df2_buy['買進股數(股)']/1000
df2_buy['名次_張數'] = df2_buy['排名']*df2_buy['買進張數']
attend = df2_buy.groupby('年月日')['買進股數(股)'].count()
attend.name = '參與交易券商數'
df2_buy = df2_buy.merge(attend, on='年月日')
df2_buy.set_index('年月日', inplace=True)

In [40]:
df2_sell['賣出張數'] = df2_sell['賣出股數(股)']/1000
df2_sell['名次_張數'] = df2_sell['排名']*df2_sell['賣出張數']
attend1 = df2_sell.groupby('年月日')['賣出股數(股)'].count()
attend1.name = '參與交易券商數'
df2_sell = df2_sell.merge(attend, on='年月日')
df2_sell.set_index('年月日', inplace=True)

In [43]:
result = pd.DataFrame(index=(df2_buy.index).unique())
result['a1'] = ((df2_buy.groupby('年月日')['買進張數'].sum())*(df2_buy.groupby('年月日')['參與交易券商數'].head(1)))/2
result['b1'] = (df2_buy.groupby('年月日')['買進張數'].sum())/2
result['c1'] = df2_buy.groupby('年月日')['名次_張數'].sum()
result['a2'] = ((df2_sell.groupby('年月日')['賣出張數'].sum())*(df2_buy.groupby('年月日')['參與交易券商數'].head(1)))/2
result['b2'] = (df2_sell.groupby('年月日')['賣出張數'].sum())/2
result['c2'] = df2_sell.groupby('年月日')['名次_張數'].sum()

In [46]:
result['買進集中度'] = (result['a1'] - (result['b1']+result['c1']))/result['a1']
result['賣出集中度'] = (result['a2'] - (result['b2']+result['c2']))/result['a2']
result['差值'] = result['買進集中度'] - result['賣出集中度']

In [47]:
result

,a1,b1,c1,a2,b2,c2,買進集中度,賣出集中度,差值
年月日,,,,,,,,,
2021-07-01,7.887563e+06,9312.3530,640848.914,7.886292e+06,9310.8530,9.344243e+05,0.917571,0.880332,0.037239
2021-07-02,7.943973e+06,9356.8590,1681699.399,7.943973e+06,9356.8590,6.103031e+05,0.787127,0.921996,-0.134869
2021-07-05,1.223778e+07,14603.5525,870643.490,1.223652e+07,14602.0525,1.289460e+06,0.927663,0.893429,0.034234
2021-07-06,5.521370e+06,6628.2955,601623.113,5.520954e+06,6627.7955,6.462877e+05,0.889837,0.881739,0.008098
2021-07-07,7.252248e+06,8664.5740,656437.933,7.252248e+06,8664.5740,9.711033e+05,0.908290,0.864902,0.043389
...,...,...,...,...,...,...,...,...,...
2022-11-15,4.961635e+07,58440.9325,8944766.789,4.956838e+07,58384.4325,1.612875e+07,0.818544,0.673438,0.145105
2022-11-16,3.730056e+07,43934.6965,9235021.279,3.725726e+07,43883.6965,9.129954e+06,0.751238,0.753770,-0.002532
2022-11-17,1.962284e+07,23167.4575,4378689.638,1.962284e+07,23167.4575,4.260306e+06,0.775677,0.781710,-0.006033


In [58]:
final = result[['買進集中度','賣出集中度','差值']].merge(result3.loc['2021-07-01':]['收盤價(元)'], left_index=True,right_index=True)

In [59]:
final

,買進集中度,賣出集中度,差值,收盤價(元)
2021-07-01,0.917571,0.880332,0.037239,578.2211
2021-07-02,0.787127,0.921996,-0.134869,573.3457
2021-07-05,0.927663,0.893429,0.034234,576.2710
2021-07-06,0.889837,0.881739,0.008098,577.2461
2021-07-07,0.908290,0.864902,0.043389,579.1962
...,...,...,...,...
2022-11-15,0.818544,0.673438,0.145105,480.0000
2022-11-16,0.751238,0.753770,-0.002532,487.0000
2022-11-17,0.775677,0.781710,-0.006033,485.0000
2022-11-18,0.822573,0.718029,0.104544,487.0000


In [63]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x = final.index, y = final['收盤價(元)'],  
               marker_color = 'rgb(220,187, 97)', name = '收盤價(元)', line = dict(width = 2)),
    secondary_y=True,) 

fig.add_trace(
    go.Scatter(x = final.index, y = final['買進集中度'],
                marker_color = 'rgb(219,113,153)', name = '買進集中度'), 
            secondary_y=False,)

fig.add_trace(
    go.Scatter(x = final.index, y = final['賣出集中度'],
                marker_color = 'rgb(97, 137, 168)', name = '賣出集中度'), 
            secondary_y=False,)

fig.add_trace(
    go.Scatter(x = final.index, y = final['差值'],
                marker_color = 'rgb(60, 179, 113)', name = '差值'), 
            secondary_y=False,)

fig.update_yaxes(title_text= '籌碼集中度') 
fig.update_yaxes(title_text= '收盤價',secondary_y=True) 

fig.update_xaxes(
          showgrid=True,
          tickformat="%Y/%m/%d",
          title_text = '日期',
          titlefont = dict(size = 15))

fig.update_layout(
    title_text="買/賣集中度與差值",
    width = 1200,
    height = 600,
    legend = dict(font = dict(size = 20)),
    legend_title = dict(font = dict(size = 20)),
    xaxis = dict(tickfont = dict(size=15)),
)

fig.show()